In this notebook I update the training set with the results of games in the past. Note that the training set is located at .../splits_optimizer/n=20modelreadydata.csv or some name similar to that. I just want to conform to that, where the shape of that dataset looks like as follow. 


In [1]:
import numpy as np
import pandas as pd
from time import sleep
from random import randint

In [2]:
model_data = pd.read_csv('../splits_optimizer/n=20model_ready_data.csv')
del model_data['Unnamed: 0']

In [3]:
model_data.shape

(4607, 30)

Looking for 30 cols? Unclear if the 'PLUS_MINUS' actually comes into play so ignore that for now, but should actually be attainable


In [4]:
model_data.head(n=3)

,road_Rolling FG,road_Rolling 3P,road_Rolling FT,road_Rolling DR,road_Rolling AST,road_Rolling ST,road_Rolling TO,road_Rolling BL,road_Rolling PTS,road_Rolling AST/TO,...,home_Rolling TO,home_Rolling BL,home_Rolling PTS,home_Rolling AST/TO,home_Rolling 3P%,home_Rolling FG%,home_Rolling FT%,home_SPREAD,PLUS_MINUS,Home Team Won?
0,41.8,9.85,16.5,36.2,27.6,7.45,13.70,5.90,109.95,2.093776,...,13.55,3.25,109.65,2.007264,0.325822,0.466812,0.772590,7.0,19,1
1,38.6,8.90,17.4,30.9,22.6,9.00,14.35,4.45,103.50,1.846354,...,13.90,4.45,106.25,1.744716,0.428645,0.459249,0.813842,-5.5,-2,0
2,38.1,8.30,18.7,29.2,23.8,9.35,15.30,5.10,103.20,1.622212,...,14.85,5.15,99.55,1.473903,0.301886,0.452531,0.670800,3.5,-15,0


In [5]:
to_update_model_data = model_data

In [6]:
#put this model dataframe in a form for updating/modeling. 

del to_update_model_data['home_SPREAD'],to_update_model_data['PLUS_MINUS'],to_update_model_data['road_SPREAD']

In [7]:
to_update_model_data

,road_Rolling FG,road_Rolling 3P,road_Rolling FT,road_Rolling DR,road_Rolling AST,road_Rolling ST,road_Rolling TO,road_Rolling BL,road_Rolling PTS,road_Rolling AST/TO,...,home_Rolling AST,home_Rolling ST,home_Rolling TO,home_Rolling BL,home_Rolling PTS,home_Rolling AST/TO,home_Rolling 3P%,home_Rolling FG%,home_Rolling FT%,Home Team Won?
0,41.80,9.85,16.50,36.20,27.60,7.45,13.70,5.90,109.95,2.093776,...,25.40,8.90,13.55,3.25,109.65,2.007264,0.325822,0.466812,0.772590,1
1,38.60,8.90,17.40,30.90,22.60,9.00,14.35,4.45,103.50,1.846354,...,22.90,6.10,13.90,4.45,106.25,1.744716,0.428645,0.459249,0.813842,0
2,38.10,8.30,18.70,29.20,23.80,9.35,15.30,5.10,103.20,1.622212,...,20.25,9.80,14.85,5.15,99.55,1.473903,0.301886,0.452531,0.670800,0
3,37.85,9.45,16.35,33.45,22.30,7.85,16.80,4.75,101.50,1.489986,...,18.30,6.45,13.15,5.05,89.60,1.544741,0.292427,0.413253,0.699586,1
4,38.30,8.00,20.05,32.40,24.25,7.95,14.05,4.50,104.65,1.830207,...,22.50,8.10,17.55,4.85,102.30,1.324584,0.313071,0.445313,0.685929,0
5,39.45,9.85,18.25,32.70,23.05,6.05,14.05,4.35,107.00,1.740389,...,19.00,7.75,15.50,4.40,92.10,1.284188,0.332995,0.432541,0.743206,0
6,38.25,8.30,18.55,28.90,23.80,9.25,15.25,4.90,103.35,1.626460,...,20.95,7.55,14.80,6.00,98.15,1.476140,0.362892,0.451514,0.798916,1
7,37.00,6.75,14.75,31.15,18.85,7.20,14.90,4.60,95.50,1.384262,...,19.10,6.85,14.00,4.35,94.40,1.448960,0.328351,0.432993,0.756132,1
8,37.75,7.75,21.45,32.05,22.75,9.50,14.30,3.25,104.70,1.729712,...,19.90,10.00,14.80,5.05,98.65,1.457237,0.300636,0.448708,0.667770,0
9,37.65,9.25,16.60,30.95,18.55,8.30,15.40,4.85,101.15,1.303920,...,23.95,7.00,14.70,6.10,100.95,1.719057,0.402230,0.441640,0.735802,0


Now I'll be working with a modified form of the functions I created in "predict today" but instead of "today" they'll be scraping arbitrary dates. 

In [8]:
#import dependencies. 
from datetime import datetime
from nba_py import Scoreboard
import numpy as np
import pandas as pd

In [9]:
#get today's date, for the games!


def get_todays_games(month,day,year):
    """
    Get the game's going on today. 
    
    Parameters:
    -----------
    
    None
    
    
    Returns 
    -------
    
    matchups : arr
        array of all the matchups today. Note I skip the first element since it just denotes road, or home which is known
        now. 
        
        
    
    """
    
    
     
    ex = Scoreboard(month=month,day=day,year=year)
    games = ex.game_header()['GAMECODE']
    matchups = []
    matchups.append(['road','home'])
    for game in games:
        away_team = game[-6:-3]
        home_team = game[-3:]

        if away_team == 'BKN':
            away_team = 'BRK'
        if home_team == 'BKN':
            home_team = 'BRK'
        
        matchups.append([away_team,home_team])
        
    return matchups[1:]

In [10]:
def create_data(month,day,year):
    """Creates a dataset identical to the one used for the ML modeling. This is done by scraping the ngames averages
    of the teams just listed, along with the spread, and cominbing. 
    
    Returns 
    -------
    
    today_matchups : arr
        In accordance with the designated format of how these team statistics will be shaped, I did that here. 
        For further explanation, please refer to the "relevant stats" and "splits_optimizer" repo's which explain why I 
        use certain values, this function simply puts them in that shape for the games I want to predict. 
    
    
    home_teams : arr
        Array of the home teams. Since I next have to obtain the spread of these games, I'll line them up based on the 
        name of the home team. 
    """
  #  today = datetime.now()
  #  day = today.day
   # month = today.month
   # year = today.year
    
    
    matchups = get_todays_games(month,day,year)
    print("# of games:",len(matchups))
    if len(matchups) < 1:
        #no games are being played today:
        todays_matchups = None
        home_teams = None
        road_teams = None
        return todays_matchups,home_teams,road_teams
    #matchups
    import numpy as np
    from nba_py import team
    teams = team.TeamList()
    teamids = teams.info()
    #print('predicting matchups of ', teamids)
  #  print()
    teamids = teamids[:-15]
   # print(teamids)
    teamids = teamids[['TEAM_ID','ABBREVIATION']]
    teamids = teamids.rename(index=str, columns={"ABBREVIATION": "Team"})
    teamids = teamids.replace('BKN','BRK')
    teamids = teamids.sort_values('Team')
   # print(teamids)
    todays_matchups = []
    home_teams = []
    road_teams = []
    for matchup in matchups:
        home_teams.append(matchup[1])
        road_teams.append(matchup[0])
        game_array = []
        for team_ in matchup:
            TEAM_ID = teamids.loc[teamids['Team'] == team_].values[0,0]
         #   print(team_,TEAM_ID)
                   ################
            date_ = str(month) +'/'+str(day) +'/'+ str(year)

            TEAM_splits = team.TeamLastNGamesSplits(team_id=TEAM_ID,season='2018-19',date_to=date_)
########## This is the one for updated dates. s
        
        
            TEAM_splits = team.TeamLastNGamesSplits(team_id=TEAM_ID,season='2018-19')
       # print(TEAM_splits.last20())
            df = TEAM_splits.last20()
        
            #retain (and create) the columns proven to be correlated to outcome. 
            df['AST/TO'] = df['AST']/df['TOV']

            df = df[['FGM','FG3M','FTM','DREB','AST','STL',
                     'TOV','BLK','PTS','AST/TO','FG3_PCT',
                     'FG_PCT','FT_PCT']]
           # df['AST/TO'] = df['AST']/df['TOV']
            game_array.append(df.values)
        
        matchup_array = np.concatenate((game_array[0],game_array[1]),axis=1)
        todays_matchups.append(matchup_array)

    #quick formating!
    todays_matchups = np.array(todays_matchups)
    todays_matchups = todays_matchups[:,0,:]
    return todays_matchups,home_teams,road_teams

In [11]:
from datetime import datetime
from nba_py import Scoreboard
import numpy as np
import pandas as pd

def get_winners(month,day,year):
    results = []
    ex = Scoreboard(month=month,day=day,year=year)
    games = ex.line_score()[['TEAM_ABBREVIATION','PTS','GAME_ID']]
    teams = []
    scores = []
    for i, game in enumerate(games.values):
        if game[0] =='BKN':  #naming convention
            game[0] = 'BRK'
        teams.append(game[0])
        scores.append(game[1])
        
    road_teams = teams[0::2]
    home_teams = teams[1::2]
    road_scores = scores[0::2]
    home_scores = scores[1::2]
 #   print("home teams order from the results function:")
 #   print(home_teams)
    for i in range(len(road_teams)):
    
        if road_scores[i] > home_scores[i]:
            results.append(0)
        else:
            results.append(1)
    home_team_order = home_teams
    return home_team_order, results

In [17]:
from datetime import date, timedelta

d1 = date(2018, 11, 29)  # start date
d2 = date(2019,1,25)  # end date

delta = d2 - d1         # timedelta
new_training_data = to_update_model_data.loc[to_update_model_data['home_Rolling DR'] == 600]
for i in range(delta.days + 1):
    sleep(randint(2,7))
    delta_date = d1 + timedelta(i)
   # print("All games that occured on ", delta_date)
   # print(get_todays_games(delta_date.month,delta_date.day,delta_date.year))
    date_ = str(delta_date.month) +'/'+str(delta_date.day) +'/'+ str(delta_date.year)
    print(date_)
    todays_matchups,home_teams,road_teams = create_data(delta_date.month,delta_date.day,delta_date.year)
    if type(todays_matchups) == type(None): #meaning there are games
        print(date_, 'has no games during it.')
        continue #should skip this instance of no games. 
        
    home_team_order, winners = get_winners(delta_date.month,delta_date.day,delta_date.year)
    #print("todays matchups", todays_matchups)
    if home_teams == home_team_order:
        print("The arrangement is still correct, phew!")
        winners = np.array(winners)
        winners = winners.reshape(len(winners),1)
        new_data = np.concatenate((todays_matchups,winners),axis=1)
        for game in new_data:
            new_training_data = new_training_data.append(dict(zip(new_training_data.columns, game)), ignore_index=True)
    else:
        print("Ugh, sorry Noah, gonna have to confirm sorting ")
       # print(home_team_order)
       #print(home_teams)
        break
print('done!')

11/29/2018
# of games: 3
The arrangement is still correct, phew!
11/30/2018
# of games: 11
The arrangement is still correct, phew!
12/1/2018
# of games: 7
The arrangement is still correct, phew!
12/2/2018
# of games: 6
The arrangement is still correct, phew!
12/3/2018
# of games: 7
The arrangement is still correct, phew!
12/4/2018
# of games: 5
The arrangement is still correct, phew!
12/5/2018
# of games: 10
The arrangement is still correct, phew!
12/6/2018
# of games: 3
The arrangement is still correct, phew!
12/7/2018
# of games: 10
The arrangement is still correct, phew!
12/8/2018
# of games: 9
The arrangement is still correct, phew!
12/9/2018
# of games: 4
The arrangement is still correct, phew!
12/10/2018
# of games: 11
The arrangement is still correct, phew!
12/11/2018
# of games: 3
The arrangement is still correct, phew!
12/12/2018
# of games: 11
The arrangement is still correct, phew!
12/13/2018
# of games: 4
The arrangement is still correct, phew!
12/14/2018
# of games: 9
The 

ConnectionError: ('Connection aborted.', OSError("(60, 'ETIMEDOUT')",))

In [24]:
y = new_training_data['Home Team Won?'].values

In [29]:
new_training_data

,road_Rolling FG,road_Rolling 3P,road_Rolling FT,road_Rolling DR,road_Rolling AST,road_Rolling ST,road_Rolling TO,road_Rolling BL,road_Rolling PTS,road_Rolling AST/TO,...,home_Rolling DR,home_Rolling AST,home_Rolling ST,home_Rolling TO,home_Rolling BL,home_Rolling PTS,home_Rolling AST/TO,home_Rolling 3P%,home_Rolling FG%,home_Rolling FT%
0,45.5,14.2,17.3,37.8,30.9,7.6,13.0,6.5,122.4,2.376923,...,33.7,23.1,8.8,13.4,5.5,111.1,1.723881,0.331,0.451,0.789
1,43.6,9.5,16.2,34.0,27.7,8.9,12.5,4.5,112.8,2.216000,...,38.9,24.9,6.6,16.3,4.9,110.3,1.527607,0.310,0.466,0.670
2,41.4,9.6,21.5,35.3,23.7,6.6,13.8,4.2,113.9,1.717391,...,33.6,26.2,8.5,13.8,4.9,109.2,1.898551,0.387,0.445,0.700
3,37.3,9.6,16.7,29.8,20.3,6.9,13.5,3.1,100.7,1.503704,...,33.4,26.7,9.4,13.3,6.1,112.8,2.007519,0.376,0.469,0.812
4,39.7,12.8,18.7,38.0,25.6,7.6,14.6,6.1,110.8,1.753425,...,35.3,22.8,6.3,14.0,4.5,108.9,1.628571,0.355,0.449,0.767
5,38.5,8.6,15.0,32.9,21.3,7.9,13.5,4.1,100.5,1.577778,...,33.1,22.2,7.2,15.1,3.3,101.6,1.470199,0.341,0.430,0.763
6,42.8,12.0,16.1,32.1,27.3,9.6,14.7,3.6,113.6,1.857143,...,37.0,28.8,7.9,15.3,6.0,117.8,1.882353,0.375,0.487,0.757
7,36.6,9.1,15.9,32.6,24.0,8.6,15.0,5.5,98.0,1.600000,...,37.1,24.4,6.2,16.1,4.3,114.7,1.515528,0.371,0.468,0.765
8,43.5,9.8,18.8,35.8,26.3,7.5,13.4,5.7,115.6,1.962687,...,34.7,25.4,7.0,14.2,5.1,103.6,1.788732,0.346,0.460,0.657
9,41.8,12.1,17.1,34.5,26.9,7.7,19.2,4.5,112.7,1.401042,...,36.2,23.7,10.2,13.8,4.7,118.5,1.717391,0.369,0.466,0.729


In [30]:
#X = new_training_data
#del X['Home Team Won?']
X = X.values

In [16]:
new_training_data.append(dict(zip(new_training_data.columns, game)), ignore_index=True)


,road_Rolling FG,road_Rolling 3P,road_Rolling FT,road_Rolling DR,road_Rolling AST,road_Rolling ST,road_Rolling TO,road_Rolling BL,road_Rolling PTS,road_Rolling AST/TO,...,home_Rolling AST,home_Rolling ST,home_Rolling TO,home_Rolling BL,home_Rolling PTS,home_Rolling AST/TO,home_Rolling 3P%,home_Rolling FG%,home_Rolling FT%,Home Team Won?
0,45.5,14.2,17.3,37.8,30.9,7.6,13.0,6.5,122.4,2.376923,...,23.1,8.8,13.4,5.5,111.1,1.723881,0.331,0.451,0.789,1.0


In [32]:
from sklearn.svm import SVC

model = SVC()
model.fit(X,y)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [33]:
model.score(X,y)

0.93181818181818177